<a href="https://colab.research.google.com/github/danzi13/ML-projects/blob/main/SpotifySongMerge-KNN_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 139 kB 15.9 MB/s 
     |████████████████████████████████| 246 kB 23.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simpl

In [ ]:
import requests
import datetime
import base64
import json
import random
import numpy as np
import pandas as pd

import spotipy.util as util
import spotipy
import yaml
from spotipy.oauth2 import SpotifyOAuth


In [ ]:
client_id = "************"
client_secret = "************"
username = "***********"

In [ ]:
scope = "user-library-read user-follow-read user-top-read playlist-read-private"
redirect_uri = "http://localhost:8888/callback"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope, 
                                               username=username,
                                               open_browser=False))



In [ ]:
playlist_id = 'https://open.spotify.com/playlist/______________'
#playlist 1 

playlist2_id = 'https://open.spotify.com/playlist/______________'
#playlist 2

#gotten from share playlist link on spotify 

In [ ]:
playlist_tracks = sp.playlist(playlist_id)
playlist2_tracks = sp.playlist(playlist2_id)

Go to the following URL: https://accounts.spotify.com/authorize?client_id=7d469221b1f342c98d50f94bcf0ffb9c&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-library-read+user-follow-read+user-top-read+playlist-read-private
Enter the URL you were redirected to: http://localhost:8888/callback?code=AQBBxkzixxEZ2sGIcsOPBlk81dVZGuYKKD6dHjHYzSZAQrw1IoiMO7Z-bJKflX8lucR8Zdy4-6BkCL_oqxC8xhq_Ox3GME7IGei_Z7BkmpNzLjHseiS2zkep9khvggGsMgGzWMxd3ON-bcPErF3sd6L5ZF0L7hLxm1Zzvv222CmaaBalBsAEyKEBKE5FXtjQRuYnqPcrHpfXEnfx_8OJkBMwAAYaS6IfiPjaO_4aLKGf9bsRJGESt9xM9O4FNMyVPvZAqVkBBaugSjJyVg


In [ ]:
songIDs, songIDs2 = [], []

#Find the name, artist, and ID for each song in the playlist

for i in range(len(playlist_tracks['tracks']['items'])):
  songIDs.append(playlist_tracks['tracks']['items'][i]['track']['id']) #Append song IDs to a list

for i in range(len(playlist2_tracks['tracks']['items'])):
  songIDs2.append(playlist2_tracks['tracks']['items'][i]['track']['id']) #Append song IDs to a list

In [ ]:
if len(songIDs) < len(songIDs2):
  min_length = len(songIDs)
else:
  min_length = len(songIDs2)

#finding min of songs in playlists

In [ ]:
random.shuffle(songIDs)
random.shuffle(songIDs2)

#shuffle for song recommendations

In [ ]:
features_list = []

#getting features of each song

for i in range(min_length):
  thing = sp.audio_features(songIDs[i])
  thing2 = sp.audio_features(songIDs2[i])
  
  for features in thing: #save each feature in the audio features json dict
    features_list.append([features['energy'], features['liveness'], features['tempo'], features['speechiness'], features['acousticness'], features['instrumentalness'],
                          features['time_signature'], features['danceability'], features['key'], features['duration_ms'], features['loudness'], features['valence'], 
                          features['mode'],0])

  for features2 in thing2: #save each feature in the audio features json dict
    features_list.append([features2['energy'], features2['liveness'], features2['tempo'], features2['speechiness'], features2['acousticness'], features2['instrumentalness'],
                          features2['time_signature'], features2['danceability'], features2['key'], features2['duration_ms'], features2['loudness'], features2['valence'], 
                          features2['mode'],1])

#Read features into a df
df = pd.DataFrame(features_list, columns=['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'key', 'duration_ms', 
                                          'loudness', 'valence', 'mode','rating'])


In [ ]:
X = df[['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 
        'time_signature', 'danceability', 'key', 'duration_ms', 
                        'loudness', 'valence', 'mode']] 

y = df['rating']

X = X.dropna()
y = y.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=20)

knn_model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=20)

In [ ]:
#StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
number = 4 #set amount of total songs gathered (number * 5 = total)

In [ ]:
complete_songID = []
features_list2 = []

for i in range(number):
  recommendations1 = sp.recommendations(seed_tracks = songIDs[i*5:5+(5*i)]) 
  recommendations2 = sp.recommendations(seed_tracks = songIDs2[i*5:5+(5*i)]) 

  for i in range(20): 
      complete_songID.append(recommendations1['tracks'][i]['id'])
      complete_songID.append(recommendations2['tracks'][i]['id'])

for i in range(len(complete_songID)): #for each song in the user's playlist
  thing = sp.audio_features(complete_songID[i])
  
  for features in thing: #save each feature in the audio features json dict
    features_list2.append([features['energy'], features['liveness'], features['tempo'], features['speechiness'], features['acousticness'], features['instrumentalness'],
                          features['time_signature'], features['danceability'], features['key'], features['duration_ms'], features['loudness'], features['valence'], 
                          features['mode']])

#Read features into a df
rec_df = pd.DataFrame(features_list2, columns=['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'key', 'duration_ms', 
                                          'loudness', 'valence', 'mode'])



In [ ]:
y_pred = knn_model.predict(rec_df)

In [ ]:
scope = "playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope, 
                                               username=username,
                                               open_browser=False))

# Create a new playlist for tracks to add - you may also add these tracks to your source playlist and proceed
new_playlist = sp.user_playlist_create(user=username, 
                                       name="spotify-merged-playlist",
                                       public=False, 
                                       collaborative=False, 
                                       description="Created by https://github.com/danzi13",
                                      )



Go to the following URL: https://accounts.spotify.com/authorize?client_id=7d469221b1f342c98d50f94bcf0ffb9c&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=playlist-modify-private
Enter the URL you were redirected to: http://localhost:8888/callback?code=AQCTrH7t5yhyNjylum0Bwbzjr31g2E8gdF7hjkZM9gO01XzL43KkSc2jLxkBLTMGPs8xmIckrE9Y4FjmzucTdGYVF1ZSSMsAscexpZMyRLXgrhjqClp7swzWmmBoR3Uh36yi-NurS4TXcxZO82az9b97cshDDzml2X8uppSxxD2L6EtBIpnynbou1Xe35dcadKnoqlSsg2NY5To


In [ ]:
for i in range(len(complete_songID)):
    #if in ML middle, add to playlist
    if(y_pred[i] >= .4):
      if(y_pred[i] <= .6):
        sp.user_playlist_add_tracks(user=username, 
                                playlist_id=new_playlist['id'], 
                                tracks=[complete_songID[i]],
                               );